## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [10]:
df = pd.read_csv('QueryResults.csv', names = ['DATE', 'TAG', 'POSTS'] )

FileNotFoundError: [Errno 2] No such file or directory: 'QueryResults.csv'

**Challenge**: Examine the first 5 rows and the last 5 rows of the of the dataframe

In [ ]:
df.head()

In [ ]:
df.tail()

**Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [ ]:
df.shape

**Challenge**: Count the number of entries in each column of the dataframe

In [ ]:
df.count()

**Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [ ]:
sum_posts = df.groupby('TAG')
sum_posts[['POSTS']].sum()

Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [ ]:
sum_posts.count()

## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [ ]:
type(df['DATE'][1])

In [ ]:
df.drop(0 ,inplace=True)
df_date = pd.to_datetime(df.DATE)

In [ ]:
# date = pd.to_datetime(df['DATE'][1])
# print(date)
# type(date)

In [ ]:
type(df_date)

## Data Manipulation



In [ ]:
pivot_df = df.pivot(index='DATE', columns='TAG', values='POSTS')

**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

In [ ]:
pivot_df.shape

In [ ]:
pivot_df.head()

**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

In [ ]:
pivot_df.count()

In [ ]:
pivot_df.fillna(0, inplace=True)
pivot_df.head()

In [ ]:
pivot_df.isna().values.any()

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart

In [ ]:
plt.figure(figsize=(16, 10))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Posts', fontsize=14)

plt.ylim(0, 35000)
plt.xlim(0, 150)

#lines on the chart:
plt.plot(pivot_df.index, pivot_df.javascript)
plt.plot(pivot_df.index, pivot_df.python)
plt.show()

# or
# plt.plot(pivot_df.index, pivot_df.javascript, pivot_df.python)


**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

In [ ]:
roll_df = pivot_df.rolling(window = 6).mean()

plt.figure(figsize=(16, 10))

plt.xticks(fontsize=14, rotation = 45)
plt.yticks(fontsize=14)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Posts', fontsize=14)

plt.ylim(0, 35000)
plt.xlim(0, 150)

for n in pivot_df.columns:
    plt.plot(pivot_df.index, roll_df[n], linewidth = 5, label = roll_df[n].name)
plt.legend(fontsize=14)

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 

In [ ]:
# with window == 12

In [ ]:
roll_df = pivot_df.rolling(window = 12).mean()

plt.figure(figsize=(16, 10))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Posts', fontsize=14)

plt.ylim(0, 35000)
plt.xlim(0, 150)

for n in pivot_df.columns:
    plt.plot(pivot_df.index, roll_df[n], linewidth = 5, label = roll_df[n].name)
plt.legend(fontsize=14)

In [ ]:
# with window == 6

In [ ]:
roll_df = pivot_df.rolling(window = 6).mean()

plt.figure(figsize=(16, 10))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Posts', fontsize=14)

plt.ylim(0, 35000)
plt.xlim(0, 150)

for n in pivot_df.columns:
    plt.plot(pivot_df.index, roll_df[n], linewidth = 5, label = roll_df[n].name)
plt.legend(fontsize=14)